In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv("../input/montcoalert/911.csv")

In [ ]:
df.info()

In [ ]:
df.head()

## Basic Data Analysis

** Top 5 zipcodes for 911 calls? **

In [ ]:
df['zip'].value_counts().head(5)

In [ ]:
# or
df.groupby('zip').count().sort_values(['lat'], ascending = False)['lat'].head(5)

** Top 5 townships (twp) for 911 calls? **

In [ ]:
df['twp'].value_counts().head(5)

** Unique title codes are there? **

In [ ]:
df['title'].nunique()

## Creating new features

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.** 

**For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. **

In [ ]:
df['Reason']= df['title'].apply(lambda x : x[:x.find(':')])

In [ ]:
df.head()

** What is the most common Reason for a 911 call based off of this new column? **

In [ ]:
df['Reason'].value_counts()

** Plot countplot using seaborn **

In [ ]:
sns.countplot(data=df, x= 'Reason')

___
** Now let us begin to focus on time information. What is the data type of the objects in the timeStamp column? **

In [ ]:
#df['timeStamp'].dtype   
type(df['timeStamp'].iloc[0])

** You should have seen that these timestamps are still strings. Use [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to convert the column from strings to DateTime objects. **

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

In [ ]:
df.dtypes    # timestamp is converted to datetime..

** You can now grab specific attributes from a Datetime object by calling them. For example:**

    time = df['timeStamp'].iloc[0]
    time.hour

**You can use Jupyter's tab method to explore the various attributes you can call. Now that the timestamp column are actually DateTime objects, use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column, reference the solutions if you get stuck on this step.**

In [ ]:
#  t= df['timeStamp'].iloc[0]
#  explore functions on t object...  t.hour, t.month, t.day_name

df['Hour']        = df['timeStamp'].apply(lambda x : x.hour)
df['Month']       = df['timeStamp'].apply(lambda x : x.month)
df['Day of Week'] = df['timeStamp'].apply(lambda x : x.dayofweek)

In [ ]:
df.head(3)

** Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week: **

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
dmap

In [ ]:
#df['Day of Week'] = df['Day of Week'].apply(lambda x : dmap[x])

In [ ]:
#or
df['Day of Week'] = df['Day of Week'].map(dmap)

In [ ]:
df.head(3)

** Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. **

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df,x='Day of Week', hue = 'Reason')
plt.legend(bbox_to_anchor=(1.05, 1),loc=2)

**Now do the same for Month:**

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(12,5))
sns.countplot(data= df, x='Month', hue='Reason')
plt.legend(bbox_to_anchor=(1.05, 1),loc=2)

** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame. **

In [ ]:
df.groupby(by='Month').count().head(3)

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
byMonth = df.groupby(by='Month').count()

In [ ]:
byMonth['lat'].plot()

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
byMonth.head()

In [ ]:
byMonth.reset_index()

In [ ]:
# need to reset the index, otherwise sns won't identify  x ='Month'  - column name..

sns.lmplot(data= byMonth.reset_index(), x= 'Month',y='twp')

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method. ** 

In [ ]:
df['Date'] = df['timeStamp'].apply(lambda datetime : datetime.date())

In [ ]:
df.head(3)

** Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
byDate = df.groupby(by='Date').count()

In [ ]:
byDate['lat'].plot()
#df.groupby(by='Date').count()['lat'].plot()
plt.tight_layout()

** Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
dfEMS     = df[df['Reason']=='EMS']
dfFire    = df[df['Reason']=='Fire'] 
dfTraffic = df[df['Reason']=='Traffic']

In [ ]:
plt_Traffic  = dfTraffic.groupby(by='Date').count()

In [ ]:
plt_Traffic['lat'].plot()
plt.tight_layout()
plt.title('Traffic')

In [ ]:
df[df['Reason']=='Fire'].groupby(by='Date').count()['lat'].plot()
plt.title('Fire')
plt.tight_layout()

In [ ]:
df[df['Reason']=='EMS'].groupby(by='Date').count()['lat'].plot()
plt.title('EMS')
plt.tight_layout()

____
** Now let's move on to creating  heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an [unstack](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method. Reference the solutions if you get stuck on this!**

In [ ]:
dfCOW = df.groupby(by=['Day of Week','Hour']).count()

In [ ]:
dfCOW   # multi-index dataframes.. 

In [ ]:
# unpacking of multi-index to columns...

In [ ]:
dfCOW['lat'].unstack()

** Now create a HeatMap using this new DataFrame. **

In [ ]:
fig = plt.figure(figsize=(12,7))
sns.heatmap(dfCOW['lat'].unstack(), cmap= 'viridis' )

** Now create a clustermap using this DataFrame. **

In [ ]:
sns.clustermap(dfCOW['lat'].unstack(), cmap= 'viridis' )

** Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. **

In [ ]:
dfDoW_month = df.groupby(by=['Day of Week','Month']).count()['lat'].unstack()

In [ ]:
dfDoW_month

In [ ]:
fig = plt.figure(figsize=(12,7))
sns.heatmap(dfDoW_month, cmap='coolwarm')

In [ ]:
sns.clustermap(dfDoW_month,cmap='coolwarm')